A notebook to complete the ELT process and display results.

In [29]:
import pandas as pd
import numpy as np
from scipy.fft import fft, fftfreq, ifft
from scipy.signal import find_peaks


class TradingBot:
    def __init__(self, source:str) -> None:
        self.source = source
        self.read_data()
        self.generate_strategy()
    def read_data(self) -> None:
        features = [
            'unix_time'
            , 'bid_price'
            , 'ask_price'
        ]
        possible_columns = [
            'unix_time'
            , 'bid_price'
            , 'bid_volume'
            , 'ask_price'
            , 'ask_volume'
        ]
        assert all([f in possible_columns for f in features]), 'Invalid columns when reading csv!'
        self.df = pd.read_csv(self.source)[features]
    def generate_strategy(self) -> None:
        '''Generating a strategy buy feature engineering trading signals.'''
        self.df['time_stamp'] = pd.to_datetime(self.df['unix_time'], unit='s')
        self.df['signal'] = 0
        self.df['capital_delta'] = 0
        self.helper_function(True)
        self.helper_function(False)
        #self.df.dropna(inplace=True)
    @property
    def strategy(self) -> pd.DataFrame:
        return self.df.copy()
    def helper_function(self, buying:bool) -> None:
        '''True=Buy, False=Sell'''
        delta_column = 'bid_delta' if buying else 'ask_delta'
        price_column = 'bid_price' if buying else 'ask_price'
        freq_column = 'bid_freq' if buying else 'ask_freq'
        amplitude_column = 'bid_amplitude' if buying else 'ask_amplitude'
        clean_amp_column = 'bid_amp_clean' if buying else 'ask_amp_clean'
        delta_clean_column = 'bid_delta_clean' if buying else 'ask_delta_clean'
        signal_symbol = -1 if buying else 1
        self.df[delta_column] = self.df[price_column].astype(float).diff()
        self.df[delta_column].fillna(0, inplace=True)
        delta_signal = self.df[delta_column].values
        n = len(delta_signal)
        timestep = (self.df['time_stamp'][1] - self.df['time_stamp'][0]).total_seconds()
        self.df[freq_column] = fftfreq(n, d=timestep)
        amplitude = np.abs(fft(delta_signal)) / (n/2)
        self.df[amplitude_column] = amplitude
        amp_mean = amplitude.mean()
        amp_std = amplitude.std()
        self.df[clean_amp_column] = np.where(amplitude > (3*amp_std+amp_mean), amplitude, 0)
        clean_fft = fft(delta_signal)
        clean_fft[self.df[clean_amp_column] == 0] = 0
        self.df[delta_clean_column] = ifft(clean_fft).real
        peaks, _ = find_peaks(-self.df[delta_clean_column])
        self.df.loc[peaks, 'signal'] = signal_symbol


class Backtest:
    def __init__(self, bot:TradingBot) -> None:
        self.strategy:pd.DataFrame = bot.strategy
    def run(self) -> None:
        quantity = -self.strategy['signal'].sum()
        self.strategy['weighted_sum'] = self.strategy['ask_price']*1 + self.strategy['bid_price']*-1
        weighted_sum = self.strategy['weighted_sum'].sum(skipna=True)
        if quantity <= 0:
            correction = quantity*self.strategy['ask_price'].iloc[-1]
        else:
            correction = quantity*self.strategy['bid_price'].iloc[-1]
        profit = int(weighted_sum + correction)
        if profit < 0:
            additional_message = 'Let\'s not try this strategy.'
        else:
            additional_message = 'Wow!'
        print(f'Strategy ended with a profit of {profit:,}. {additional_message}')


def main() -> None:
    data_stream = 'bond_timeseries.csv'
    bot = TradingBot(data_stream)
    test = Backtest(bot)
    test.run()
    #print(bot.strategy.info())
    #print(bot.strategy.iloc[:15,0:5])

if __name__ == '__main__':
    main()

Strategy ended with a profit of -604,930,945. Let's not try this strategy.
